In [1]:
import numpy as np
import pandas as pd

In [2]:
# # 读入训练和测试数据
# ## 本地，同时平衡正反例
# def balance(train_pro):
#     ## 从正例中随机取出比例为a的数据，划分
#     data = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\train.csv')
#     train_size_p = int(data[data['label']==1]['label'].count()*train_pro)
#     train_size_n = int(train_size_p * 39.20424181) #原始样本中负例是正例的这个倍数(所有样本)
# #     train_size_n = int(train_size_p * 38.16572737) #原始样本中负例是正例的这个倍数（剔除了29和30号的样本）
#     sampler1 = np.random.permutation(data[data['label']==1]['label'].count())
#     index1 = data[data['label']==1].index[sampler1] #重排序后的索引
#     train1 = data[data['label']==1].ix[index1[:train_size_p]]
#     test1 = data[data['label']==1].ix[index1[train_size_p:]]
#     ## 从负例中随机取出相同数量的数据，划分
#     sampler0 = np.random.permutation(data[data['label']==0]['label'].count())
#     index0 = data[data['label']==0].index[sampler0] #重排序后的索引
#     train0 = data[data['label']==0].ix[index0[:train_size_n]]
#     test0 = data[data['label']==0].ix[index0[train_size_n:]]
#     ## 构建训练集和测试集
#     data_train = pd.concat([train1, train0])
#     data_test = pd.concat([test1, test0])
#     return data_train, data_test
# # 获取正反例
# trainData, testData = balance(0.8)

In [3]:
# 预测
trainData = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\train.csv')
testData = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\test.csv')

In [4]:
# 读入基础数据
for i,dt in enumerate([trainData,testData]):
    dt['date'] = dt['clickTime'].apply(lambda x: int(x/10000))
    dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\user.csv'), on='userID')
    dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\ad.csv'), on='creativeID')
    dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\position.csv'), on='positionID')
    if i == 0:
        trainData = dt
    elif i == 1:
        testData = dt

In [5]:
# 本地测试与训练
testData = trainData[trainData['date']==28]
trainData = trainData[trainData['date']<28]

In [6]:
# 对点击时间进行处理
def dealTime(clicktime):
    hour = int(str(clicktime)[2:4])
    minute = int(str(clicktime)[4:6])
    x = int(60*hour+minute)/30
    if x in [18,19,20]:
        return 1
    elif x in [21,22,23]:
        return 2
    elif x in [4,24]:
        return 3
    elif x in [25,26]:
        return 4
    elif x in [2,3,6,10,27,31,47]:
        return 5
    elif x in [0,1,5,7,8,9,11,28,30,44,45]:
        return 6
    elif x in [13,14]:
        return 8
    elif x in [15]:
        return 9
    else: #中下的最多，所以else来处理
        return 7

In [7]:
# 读入更多特征
for i,dt in enumerate([trainData,testData]):
    ## 读入age
    dt.drop(['age'], axis=1, inplace=True) #有些原有字段中有的但经过处理的，先删除，再读入
    dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\age.csv'), on='userID')
    ## residence提取省份
    # dt['residence'] = dt['residence'].apply(lambda x: int(str(x)[-4:-2]) if x>0 else 0)
    ## 读入appCategory
    dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\appCategory.csv'), on='creativeID')
    ## 读入clickTime_hour
    dt['clickTime_hour'] = dt['clickTime'].apply(dealTime)
    ## 读入if_appinstall
    dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\if_appinstall.csv'), on='userID')
    ## 读入if_appaction
    dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\if_appaction.csv'), on='userID')
    ## 读入count_appinstall
    dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\count_appinstall.csv'), on='userID')
    ## 读入count_appaction
    dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\count_appaction.csv'), on='userID')
    ## 读入lastNCVR_advertiserID
    dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\lastNCVR_advertiserID.csv'), on=['date', 'creativeID'])
    ## 读入lastNCVR_appID
    dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\lastNCVR_appID.csv'), on=['date', 'creativeID'])
    ## 读入lastNCVR_camgaignID
    dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\lastNCVR_camgaignID.csv'), on=['date', 'creativeID'])
    ## 读入lastNCVR_positionType
    dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\lastNCVR_positionType.csv'), on=['date', 'positionID'])
    ## 读入lastNCVR_sitesetID
    dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\lastNCVR_sitesetID.csv'), on=['date', 'positionID'])
    if i == 0:
        trainData = dt
    elif i == 1:
        testData = dt

### 1.读入基本特征

In [9]:
# 连续特征标准化
import sklearn.preprocessing as preprocessing
# features = ['lastNCVR_appID', 'lastNCVR_advertiserID','lastNCVR_positionType', 'lastNCVR_sitesetID','lastNCVR_camgaignID',
#            'count_appinstall','count_appaction']
features= ['count_appaction']
scaler = preprocessing.StandardScaler()
for i,feature in enumerate(features):
    x_train = scaler.fit_transform(trainData[feature].values.reshape(-1,1))
    x_test = scaler.transform(testData[feature].values.reshape(-1,1))
    if i == 0:
        X_train, X_test = x_train, x_test
    else:
        X_train, X_test = np.hstack((X_train, x_train)), np.hstack((X_test, x_test))

In [ ]:
# 离散变量onehot化
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
enc = OneHotEncoder()
features = ['connectionType', 'telecomsOperator', 'marriageStatus','haveBaby', 'appPlatform', 
            'education', 'sitesetID', 'appCategory', 'clickTime_hour','appID','age','gender','positionType',
            'if_appinstall','if_appaction']
#             'creativeID', 'adID', 'camgaignID', 'advertiserID', 'positionID']
for feature in features:
    x_train = enc.fit_transform(trainData[feature].values.reshape(-1, 1))
    x_test = enc.transform(testData[feature].values.reshape(-1, 1))
    X_train, X_test = sparse.hstack((X_train, x_train)), sparse.hstack((X_test, x_test))

### 2.读入GBDT特征

In [ ]:
# 连续特征标准化
import sklearn.preprocessing as preprocessing
features = ['lastNCVR_appID', 'lastNCVR_advertiserID','lastNCVR_positionType', 'lastNCVR_sitesetID','lastNCVR_camgaignID']
scaler = preprocessing.StandardScaler()
for i,feature in enumerate(features):
    x_train_gbdt = scaler.fit_transform(trainData[feature].values.reshape(-1,1))
    x_test_gbdt = scaler.fit_transform(testData[feature].values.reshape(-1,1))
    if i == 0:
        X_train_gbdt, X_test_gbdt = x_train_gbdt, x_test_gbdt
    else:
        X_train_gbdt, X_test_gbdt = np.hstack((X_train_gbdt, x_train_gbdt)), np.hstack((X_test_gbdt, x_test_gbdt))

In [ ]:
# 不onehot的离散变量
# features = ['creativeID', 'adID', 'camgaignID', 'advertiserID']
features = ['connectionType', 'telecomsOperator', 'marriageStatus','haveBaby', 'appPlatform', 
            'education', 'sitesetID', 'appCategory', 'clickTime_hour','appID','age','positionType',
            'advertiserID', 'adID','if_appinstall','if_appaction']
for feature in features:
    x_train_gbdt = trainData[feature].values.reshape(-1,1)
    x_test_gbdt = testData[feature].values.reshape(-1,1)
    X_train_gbdt, X_test_gbdt = np.hstack((X_train_gbdt, x_train_gbdt)), np.hstack((X_test_gbdt, x_test_gbdt))

In [ ]:
# 读取gbdt分类器
from sklearn.externals import joblib
grd = joblib.load('C:\\Users\\work\\Desktop\\tencent\\clf\\gbdt.pkl')

In [ ]:
# 获得gbdt特征
grd_enc = OneHotEncoder() #one-hot编码 
X_train_gbdt = grd_enc.fit_transform(grd.apply(X_train_gbdt)[:, :, 0]) #每个数据，在每棵树，叶节点index
X_test_gbdt = grd_enc.transform(grd.apply(X_test_gbdt)[:, :, 0])
X_train = sparse.hstack((X_train, X_train_gbdt))
X_test = sparse.hstack((X_test, X_test_gbdt))

### 3.读入y值

In [10]:
# y值
y_train = trainData['label'].values
y_test = testData['label'].values

# 训练分类器

In [11]:
# 划分训练集和测试集
from sklearn import cross_validation
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
# 训练
clf = linear_model.LogisticRegressionCV(Cs=10, penalty='l2', tol=1e-4, n_jobs=-1, cv=5)
clf.fit(X_train, y_train)
print clf

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)


In [14]:
# 评估模型结果
import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll
from sklearn import metrics
predict = clf.predict(X_test)
print 'P&R'
print metrics.classification_report(y_test, predict)
print 'loss:'
predict_pro = clf.predict_proba(X_test)[:, 1]
# predict_pro = predict_pro - predict_pro.min()
print logloss(y_test, predict_pro)

P&R
             precision    recall  f1-score   support

          0       0.97      1.00      0.99   2762579
          1       0.00      0.00      0.00     72149

avg / total       0.95      0.97      0.96   2834728

loss:
0.117777937359


In [13]:
print y_test[10:20]
print predict_pro[10:20]

[0 0 0 0 0 0 0 0 0 0]
[ 0.02242665  0.04746538  0.02242665  0.02242665  0.02242665  0.02242665
  0.02242665  0.02242665  0.02242665  0.02242665]


In [ ]:
cf = pd.DataFrame({'columns':data.columns[1:], 'importance':list(grd.feature_importances_.T)})
# cf = pd.DataFrame({'importance':list(grd.feature_importances_.T)})
cf.sort_values(by=['importance'],ascending=False, inplace=True)
print cf

In [ ]:
cf.to_csv('C:\\Users\\work\\Desktop\\tencent\\clf\\coef_6.csv',index=False)

In [ ]:
cf

In [ ]:
# 保存分类器
from sklearn.externals import joblib
joblib.dump(grd_lm, 'C:\\Users\\work\\Desktop\\tencent\\clf\\lr_0.pkl', compress=3)

# 预测

In [ ]:
# 读取分类器
from sklearn.externals import joblib
clf = joblib.load('C:\\Users\\work\\Desktop\\tencent\\clf\\1.pkl')

In [ ]:
# 提取特征并预测
## 只选择instanceID>0的记录，并重排序instanceID
dataPredict = data[data.instanceID>0].sort_values(by='instanceID')

In [ ]:
dataPredict.head()

In [ ]:
X = dataPredict.as_matrix()[:,2:]
y = clf.predict_proba(X)

In [ ]:
output = pd.DataFrame({'instanceID':range(1,dataPredict['instanceID'].count()+1),'prob':y[:, 1]})

In [ ]:
output.to_csv('C:\\Users\\work\\Desktop\\tencent\\predict\\5.csv', index=False)

In [ ]:
d = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\predict\\5.csv')

In [ ]:
d.head()